In [278]:
! pip install xlwings 
! pip install xlsxwriter 
import xlsxwriter
import xlwings as xw
import numpy as np
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [279]:
workbook  = xlsxwriter.Workbook('recipes_model.xlsx')
worksheet = workbook.add_worksheet('Модель')

columns  = ["Сущность", "Атрибут", "Тип данных", "Обязательно к заполнению", "Описание сущности", "Описание атрибута", "Ключ", "Таблица для ссылки", "Поле для ссылки", "SQL"]

for i in range(len(columns)):
    worksheet.write(0, i, columns[i])

    
#workbook.close()

In [280]:
format1 = workbook.add_format({'bg_color':   '#006600',
                               'font_color': '#fffff'})
format2 = workbook.add_format({'bg_color':   '#cccc00',
                               'font_color': '#000000'})
worksheet.conditional_format('G2:G50', {'type':     'text',
                                              'criteria': 'containing',
                                              'value':    'PK',
                                              'format':   format1})
worksheet.conditional_format('G2:G50', {'type':     'text',
                                              'criteria': 'containing',
                                              'value':    'FK',
                                              'format':   format2})

workbook.close()

In [281]:
import csv
with open("recipes_model (1).csv", "r", encoding="utf8") as fp:
    ff = csv.reader(fp, delimiter="\t")
#     for row in ff:
#         print(row)
    a =  list(ff)    
a    

[['recipe',
  'name',
  'varchar(100)',
  'Y',
  'Рецепт',
  'Название рецепта',
  '',
  '',
  ''],
 ['recipe', 'id', 'serial', 'N', 'Рецепт', 'ID рецепта', 'PK', '', ''],
 ['recipe',
  'minutes',
  'int',
  'Y',
  'Рецепт',
  'Длительность выполнения в минутах',
  '',
  '',
  ''],
 ['recipe',
  'contributor_id',
  'int',
  'Y',
  'Рецепт',
  'ID автора рецепта',
  'FK',
  'user',
  'id'],
 ['recipe',
  'submitted',
  'timestamp',
  'Y',
  'Рецепт',
  'Дата добавления',
  '',
  '',
  ''],
 ['recipe',
  'n_steps',
  'int',
  'N',
  'Рецепт',
  'Кол-во шагов рецепта',
  '',
  '',
  ''],
 ['recipe',
  'description',
  'text',
  'N',
  'Рецепт',
  'Описание рецепта',
  '',
  '',
  ''],
 ['recipe',
  'n_ingredients',
  'int',
  'N',
  'Рецепт',
  'Кол-во ингредиентов в рецепта',
  '',
  '',
  ''],
 ['review', 'id', 'serial', 'Y', 'Отзыв', 'ID отзыва', 'PK', '', ''],
 ['review', 'user_id', 'int', 'Y', 'Отзыв', 'ID автора отзыва', 'FK', '', ''],
 ['review',
  'recipe_id',
  'int',
  'Y',
  'О

In [282]:
import xlwings as xw

workbook  = xw.Book('recipes_model.xlsx')
worksheet = workbook.sheets["Модель"]
worksheet.range('A2').value  = a

In [283]:
worksheet.range(f"J2:J{len(a)+1}").formula = '=CONCATENATE(B2, " ", UPPER(C2), IF(G2="PK", " PRIMARY KEY", \
                                                            IF(G2="FK", CONCATENATE(" REFERENCES ", H2, "(", I2, ")"), "")), \
                                                            IF(AND(D2="Y", G2<>"PK"), " NOT NULL", ""))'

При помощи пакета xlwings измените стилизацию листа Модель.
для заголовков добавьте заливку цвета 00ccff
примените автоподбор ширины столбца;
сделайте шрифт заголовков полужирным;
добавьте таблице автофильтр.
Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [284]:
worksheet.range("A1:J1").color = "00ccff"
worksheet.autofit(axis="columns")
worksheet['A1'].api.AutoFilter(Field=1)

True

В данных, полученных из файла recipes_model.csv, имеются ошибки двух видов:

1. Для некоторых полей, указанных как первичный ключ, в столбце "Обязательно к заполнению" стоит значение "N";

2. Для некоторых полей, указанных как внешний ключ, не указана таблица для ссылки и поле для ссылки.

Получите данные из листа в виде pd.DataFrame. Найдите строки на листе, которые содержат ошибки первого или второго рода и сделайте заливку эти строк цвета (255, 102, 102). Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [346]:
aa = worksheet.range('A1:J18').value
df1 = pd.DataFrame(aa)
df1.columns = df1.iloc[0]
df1 = df1.drop(labels = [0], axis = 0)
type1Error = (df1['Ключ'] == 'PK') & (df1['Обязательно к заполнению'] == 'N')
type2Error = (df1['Ключ'] == "FK") & ((df1['Таблица для ссылки'].isnull()) | (df1['Поле для ссылки'].isnull()))

q1 = (str((df1[type2Error].index[0]))).split(',')
q2 = list(str((df1[type1Error].index[0])))
some_rgb_vale = (255, 102, 102)
for i in q1:
    worksheet.range(f'A{int(i)}:J{int(i)}').color = some_rgb_vale
for j in q2:
    worksheet.range(f'A{int(j)}:J{int(j)}').color = some_rgb_vale

При помощи пакета xlwings создайте лист с названием Ошибки заполнения.
Поместите в первой строчке этого листа заголовки "№ строки" и "Ошибка". Заполните лист на основе решения задачи 6. В столбце "№ строки" укажите номер строки с листа Модель, содержащую ошибку, а в столбце "Ошибка" для ошибок первого рода напишите сообщение "Первичный ключ не может быть NULL", для ошибок второго рода - "Не указано поле для связи по внешнему ключу". Сделайте скриншот состояния листа Ошибки заполнения и прикрепите в ячейку ноутбука.

In [347]:
worksheet_new = workbook.sheets.add(name="Ошибки заполнения")

In [348]:
worksheet_new.range('A1').value = ["№ строки", "Ошибка"]

In [355]:
a = 0
b = 0
for i in q2:
    worksheet_new.range(f'A{2+a}:B{2+a}').value = [i, 'Первичный ключ не может быть NULL']
    a += 1 
for i in q1:
    worksheet_new.range(f'A{2+a}:B{2+a}').value = [i, 'Не указано поле для связи по внешнему ключу']
    a += 1 

8. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 

In [368]:
df1.groupby('Сущность')['Атрибут'].count()
worksheet_new3 = workbook.sheets.add(name='Статичтика')
worksheet_new3.range('A1').value = df1.groupby('Сущность')['Атрибут'].count()

In [402]:
import matplotlib.pyplot as plt
import xlwings as xw
plt_ = plt.figure()
y = df1.groupby('Сущность')['Атрибут'].count()
list(y)
x = y.index
x
plt.bar(x, y)
plt_

worksheet_new3.pictures.add(plt_, name='BarDiag', left=worksheet_new3.range('G15').left)

<Picture 'BarDiag' in <Sheet [recipes_model.xlsx]Статичтика>>